In [1]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
from numpy import gradient
from numpy.linalg import norm
import pickle
from packages.helper import traj_speed
from packages import data_container
from packages.data_container import Data
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

In [4]:
'''Import experiment 1 data'''

header=('subj_x', 'subj_y', 'subj_yaw')
info={'p_goal':[], 'p_obst':[], 'obst_angle':[], 'obst_speed':[], 'subj_id':[], 'trial_id':[], 'stimuli_onset':[], 
      'w_goal': 0.1, 'w_obst': 0.1, 'order':[], 'ps':[]}
exp1 = Data(Hz=30, header=header, info=info)

file_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'data\\Cohen_movObst_rawData\\Cohen_movObst_exp1_rawData.csv'))

ii = 0
head = 0
tail = 0
# subj_ps = {}
ps = 0
with open(file_path, 'r') as f:
    trials = [line.split(',') for line in f.read().split('\n')]
    head = 0
    while True:
        i_subj = int(trials[head][1])
        i_trial = int(trials[head][3])
        order = int(trials[head][5])
        for i in range(head+1, len(trials)):
            if trials[i][0] == '':
                tail = i - 1
                break
        subj = tuple([(float(line[5])/100, float(line[6])/100, float(line[7])) for line in trials[head+2:tail+1]])
        goal = tuple([(float(line[1])/100, float(line[2])/100) for line in trials[head+2:tail+1]])
        obst = np.array([[float(line[3])/100, float(line[4])/100] for line in trials[head+2:tail+1]])
        onset = 0         
        for i in range(len(subj)):
            if subj[i][1] >= 1:
                onset = i
                break
                
        exp1.add_traj(subj)
        subj_filtered = np.array(exp1.get_traj(len(exp1.trajs)-1))[:,:2]
        # Use speed at -1 second as the trial preferred speed
        speed = traj_speed(subj_filtered, exp1.Hz)
        if len(speed) > 1 * exp1.Hz:
            ps = speed[-1*exp1.Hz]
        else:
            ps = 0
        # Use the speed at the onset of stimuli as the preferred speed of the trial
#         if i_subj in subj_ps:
#             subj_ps[i_subj].append(speed[onset])
#         else:
#             subj_ps[i_subj] = [speed[onset]]        
            
        # After inspection, those trials are no good
        if len(obst) < 32 or subj[-1][1] < 0:
            exp1.add_dump(ii, 'recording error')

        if norm(obst[-1,:] - obst[-31,:]) < 0.42:
            s_obst = 0.4
        elif norm(obst[-1,:] - obst[-31,:]) < 0.62:
            s_obst = 0.6
        elif norm(obst[-1,:] - obst[-31,:]) < 0.82:
            s_obst = 0.8
        else:
            s_obst = None

        if obst[0,1] < 4.1:
            angle = 70
        elif obst[0,1] < 4.8:
            angle = 90
        elif obst[0,1] < 5.4:
            angle = 110
        else:
            angle = None
            
        if obst[0,0] < 0 and angle:
            angle = -angle
            
        obst = tuple([tuple(p) for p in obst])        
        
        exp1.add_info({'p_goal':goal, 'p_obst':obst, 'obst_angle':angle, 'obst_speed':s_obst, 'subj_id':i_subj, 
                       'trial_id':i_trial, 'stimuli_onset':onset, 'ps':ps})
        if tail + 3 >= len(trials):
            break
        head = tail + 2
        ii += 1
        
# Setting subject preferred speed to every trial
# for i in range(len(exp1.trajs)):
#     i_subj = exp1.info['subj_id'][i]
#     exp1.add_info({'ps': np.mean(subj_ps[i_subj])})

outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'data\\Cohen_movObst_exp1_data.pickle'))        
with open(outfile, 'wb') as file:   
    pickle.dump(exp1, file, pickle.HIGHEST_PROTOCOL)       

In [7]:
'''Check imported data'''
%matplotlib qt
vals = []
for i in range(len(exp1.trajs)):
    if i not in exp1.dump:
        onset = exp1.info['stimuli_onset'][i]
        p_subj = np.array(exp1.trajs[i])
        p_goal = np.array(exp1.info['p_goal'][i])
        p_obst = np.array(exp1.info['p_obst'][i])
        angle = exp1.info['obst_angle'][i]
        vals.append(exp1.info['ps'][i])
        speed = traj_speed(np.array(exp1.get_traj(i))[:,0:2], exp1.Hz)
        print(angle)
#         plt.plot(speed)
#         plt.plot(vals)
#         if angle == 70 or angle == -70:
#             plt.plot(p_obst[:,0], p_obst[:,1])

#         print(onset, p_subj[onset][1])


In [5]:
'''Import experiment 2 data'''

header=('subj_x', 'subj_y', 'subj_yaw')
info={'p_goal':[], 'p_obst':[], 'obst_angle':[], 'obst_dist':[], 'subj_id':[], 'trial_id':[], 'stimuli_onset':[],
     'w_goal': 0.1, 'w_obst': 0.1, 'ps':[]}
exp2 = Data(Hz=30, header=header, info=info)

file_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'data\\Cohen_movObst_rawData\\Cohen_movObst_exp2_rawData.csv'))
ii = 0
head = 0
tail = 0
# subj_ps = {}
ps = 0
with open(file_path, 'r') as f:
    trials = [line.split(',') for line in f.read().split('\n')]
    head = 0
    while True:
        i_subj = int(trials[head][1])
        i_trial = int(trials[head][3])
        order = int(trials[head][5])
        for i in range(head+1, len(trials)):
            if trials[i][0] == '':
                tail = i - 1
                break
        subj = tuple([(float(line[5])/100, float(line[6])/100, float(line[7])) for line in trials[head+2:tail+1]])
        goal = tuple([(float(line[1])/100, float(line[2])/100) for line in trials[head+2:tail+1]])
        obst = np.array([[float(line[3])/100, float(line[4])/100] for line in trials[head+2:tail+1]])
        onset = 0         
        for i in range(len(subj)):
            if subj[i][1] >= 1:
                onset = i
                break
        exp2.add_traj(subj)
        # Use speed at -1 second as the trial preferred speed
        speed = traj_speed(np.array(exp2.get_traj(len(exp2.trajs)-1))[:,:2], exp2.Hz)
        if len(speed) > 1 * exp2.Hz:
            ps = speed[-1*exp2.Hz]
        else:
            ps = 0
        # Use the speed at the onset of stimuli as the preferred speed of the trial
#         if i_subj in subj_ps:
#             subj_ps[i_subj].append(speed[onset])
#         else:
#             subj_ps[i_subj] = [speed[onset]]
            
        # After inspection, those trials are no good
        if len(obst) < 30:
            exp2.add_dump(ii, 'recording error')
        if ii == 325 or ii == 778:
            exp2.add_dump(ii, 'abnormal speed')
            
        if obst[0, 1] < 5:
            dist = None
        elif obst[0, 1] < 6:
            dist = 'near'
        elif obst[0, 1] < 7:
            dist = 'far'
        
        if obst[-9][1] - obst[-1][1] > 0.1:
            angle = 140
        elif obst[-9][1] - obst[-1][1] > 0.08:
            angle = 130
        elif obst[-9][1] - obst[-1][1] > 0.06:
            angle = 120
        elif obst[-9][1] - obst[-1][1] > 0.04:
            angle = 110
        elif obst[-9][1] - obst[-1][1] > 0.02:
            angle = 100
        else:
            angle = None
        if obst[0,0] < 0 and angle:
            angle = -angle
        obst = tuple([tuple(p) for p in obst])
        exp2.add_info({'p_goal':goal, 'p_obst':obst, 'obst_angle':angle, 'obst_dist':dist, 'subj_id':i_subj, 
                       'trial_id':i_trial, 'stimuli_onset':onset, 'ps':ps})
        if tail + 3 >= len(trials):
            break
        head = tail + 2
        ii += 1
        
# Setting subject preferred speed to every trial
# for i in range(len(exp2.trajs)):
#     i_subj = exp2.info['subj_id'][i]
#     exp2.add_info({'ps': np.mean(subj_ps[i_subj])})
    
outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'data\\Cohen_movObst_exp2_data.pickle'))          
with open(outfile, 'wb') as file:   
    pickle.dump(exp2, file, pickle.HIGHEST_PROTOCOL)  

In [182]:
'''Check imported data'''
%matplotlib qt
vals = []
dump = []
speed = traj_speed(np.array(exp2.get_traj(778))[:,0:2], exp2.Hz)
plt.plot(speed)
for i in range(len(exp2.trajs)):
    if i not in exp2.dump:
        onset = exp2.info['stimuli_onset'][i]
        i_subj = exp2.info['subj_id'][i]
        p_subj = np.array(exp2.trajs[i])
        p_goal = np.array(exp2.info['p_goal'][i])
        p_obst = np.array(exp2.info['p_obst'][i])        
        angle = exp2.info['obst_angle'][i]
        dist = exp2.info['obst_dist'][i]
        
        speed = traj_speed(np.array(exp2.get_traj(i))[:,0:2], exp2.Hz)
#         plt.plot(speed)
#         if i == 325 or i == 778:
#             plt.plot(speed)
#         if sum(speed > 3) > 0:
#             print(i)
#         vals.append(exp2.info['ps'][i])
#         plt.plot(vals)

#         print(onset, p_subj[onset][1])